# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [ ]:
!pip install chromadb

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
llm = OpenAI(temperature=0)

In [13]:
# This code attempts to find the path to a "state_of_the_union.txt" file by:
# 1. Starting from current directory and getting its absolute path parts
# 2. Building up path parts until it finds "langchain/docs/modules" sequence
# 3. Using those parts to construct path to the text file
# However, this approach seems unnecessary since we just need the file directly
from pathlib import Path
"""
relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")
"""
## lOAD SONNET.TXT
doc_path = 'D:/IRON_HACK/Week6_LangChain/Lessons/archive/sonnets.txt'


In [5]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

In [6]:
#state_of_union = RetrievalQA.from_chain_type(
#    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
#)

sonnets = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [7]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [9]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2122, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1017, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [10]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [16]:
tools = [
    #Tool(
    #    name="State of Union QA System",
    #    func=state_of_union.run,
    #    description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    #),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
    Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about sonnets. Input should be a fully formed question.",
    ),
]

In [17]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [18]:
#agent.invoke(
#    "What did biden say about ketanji brown jackson in the state of the union address?"
#)

agent.invoke("What is a sonnet and what are its key characteristics?")




> Entering new AgentExecutor chain...
 A sonnet is a type of poem with a specific structure and rhyme scheme.
Action: Sonnets QA System
Action Input: "What is a sonnet and what are its key characteristics?"
Observation:  A sonnet is a type of poem that originated in Italy and is typically composed of 14 lines. It follows a specific rhyme scheme and structure, with a volta or turn in the middle. Sonnets often explore themes of love, beauty, and mortality. They are known for their use of figurative language, such as metaphors and similes, and their use of iambic pentameter. Sonnets are also known for their brevity and their ability to convey complex emotions and ideas in a concise manner.
Thought: This is a comprehensive answer, but I want to confirm it with another source.
Action: Ruff QA System
Action Input: "What are the key characteristics of a sonnet?"
Observation:  I don't know.
Thought: It seems like the Ruff QA System is not as knowledgeable about poetry as the Sonnets QA Syste

{'input': 'What is a sonnet and what are its key characteristics?',
 'output': 'A sonnet is a type of poem with 14 lines, a specific rhyme scheme, and a volta or turn. It often explores a single theme and uses poetic devices such as metaphors and imagery. Sonnets are also known for their use of iambic pentameter and are traditionally written in a specific form.'}

In [14]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one would choose ruff over flake8.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and type checkers like Mypy. This means that you can use Ruff alongside these tools to get more comprehensive feedback on your code.

3. Automatic fixing of lint violations: Unlike Flake8, Ruff is capable of automatically fixing its own lint violations. This can save you time and effort when fixing issues in your code.

4. Native implementation of popular Flake8 plugins: Ruff re-imp

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, better compatibility with other tools, automatic fixing of lint violations, and native implementation of popular Flake8 plugins, making it a more comprehensive and user-friendly option compared to Flake8.'}

## Notes:
- Until this part we have two RetrevialQA from two database: 
1.  text file sonnets
2. ruff websearch. 
- Then we created a two tools from these RETRIEVAL FUNTIONS a one agent that will be able to answer questions from thess two databases. 

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [22]:
tools = [
    #Tool(
    #    name="State of Union QA System",
    #    func=state_of_union.run,
    #    description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    #    return_direct=True,
    #),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
     Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about sonnets. Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [23]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
#agent.invoke("What did biden say about ketanji brown jackson in the state of the union address?")

agent.invoke("What is a sonnet and what are its key characteristics?")



> Entering new AgentExecutor chain...
 A sonnet is a type of poem with a specific structure and rhyme scheme.
Action: Sonnets QA System
Action Input: "What is a sonnet and what are its key characteristics?"
Observation:  A sonnet is a type of poem that originated in Italy and is typically composed of 14 lines. It follows a specific rhyme scheme and structure, with a volta or turn in the last two lines. Sonnets often explore themes of love, beauty, and mortality. They are known for their use of poetic devices such as metaphor, imagery, and personification. Some key characteristics of a sonnet include its length, structure, and use of poetic devices.


> Finished chain.


{'input': 'What is a sonnet and what are its key characteristics?',
 'output': ' A sonnet is a type of poem that originated in Italy and is typically composed of 14 lines. It follows a specific rhyme scheme and structure, with a volta or turn in the last two lines. Sonnets often explore themes of love, beauty, and mortality. They are known for their use of poetic devices such as metaphor, imagery, and personification. Some key characteristics of a sonnet include its length, structure, and use of poetic devices.'}

In [25]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one would choose ruff over flake8.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: 
There are a few reasons why someone might choose to use Ruff over Flake8:

1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.

2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.

3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.

4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.



{'input': 'Why use ruff over flake8?',
 'output': '\nThere are a few reasons why someone might choose to use Ruff over Flake8:\n\n1. Larger rule set: Ruff implements over 800 rules, while Flake8 only implements around 200. This means that Ruff can catch more potential issues in your code.\n\n2. Better compatibility with other tools: Ruff is designed to work well with other tools like Black, isort, and yesqa, while Flake8 may have conflicts with these tools.\n\n3. Automatic fixing of lint violations: Ruff is capable of automatically fixing its own lint violations, while Flake8 does not have this capability.\n\n4. Native implementation of popular Flake8 plugins: Ruff natively implements some of the most popular Flake8 plugins, making it more efficient and less prone to conflicts.\n\n5. Can be used as a formatter: Ruff can be used as a formatter, making it a more versatile tool than Flake8.\n\nUltimately, the choice between Ruff and Flake8 will depend on the specific needs and preferences

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [26]:
tools = [
    #Tool(
    #    name="State of Union QA System",
    #    func=state_of_union.run,
    #    description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    #),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
     Tool(
        name="Sonnets QA System",
        func=sonnets.run,
        description="useful for when you need to answer questions about sonnets. Input should be a fully formed question.",
    ),
]

In [27]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [28]:
#agent.invoke("What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?")

# 
agent.invoke("What are the similarities between sonnets and ruff? Compare their structure and organization.")



> Entering new AgentExecutor chain...
 I should use the Sonnets QA System to answer this question.
Action: Sonnets QA System
Action Input: "What are the similarities between sonnets and ruff? Compare their structure and organization."
Observation:  Sonnets and ruffs both have a structured and organized form. Sonnets typically have 14 lines and follow a specific rhyme scheme, while ruffs have a specific shape and are made up of pleated fabric. Both sonnets and ruffs also have a specific purpose or message, with sonnets often expressing love or admiration and ruffs being a fashion statement. Additionally, both sonnets and ruffs were popular during the Renaissance period.
Thought: I should also use the Ruff QA System to gather more information about ruffs specifically.
Action: Ruff QA System
Action Input: "What is the purpose of ruffs?"
Observation:  Ruff is a linter and formatter for Python code. It is designed to help developers catch and fix errors, enforce coding standards, and impr

{'input': 'What are the similarities between sonnets and ruff? Compare their structure and organization.',
 'output': 'The similarities between sonnets and ruffs include their structured and organized form, specific purpose or message, and popularity during the Renaissance period. However, sonnets are a form of poetry while ruffs are a tool for improving code quality in Python.'}

In [29]:

agent.invoke("What tool does ruff use to run over Jupyter Notebooks? Who wrote the most popular sonnet?")



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer the first question and the Sonnets QA System to answer the second question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff does not currently have a specific tool for running over Jupyter Notebooks. However, it is possible to use Ruff with Jupyter Notebooks by converting the notebook to a Python script and then running Ruff on the script.
Thought: I should now use the Sonnets QA System to answer the second question.
Action: Sonnets QA System
Action Input: Who wrote the most popular sonnet?
Observation:  I don't know.
Thought: I should try rephrasing the question.
Action: Sonnets QA System
Action Input: Who is the author of the most famous sonnet?
Observation:  The most famous sonnet is "Sonnet 18" by William Shakespeare.
Thought: I now know the final answer.
Final Answer: The most famous sonnet is "Sonnet 18" by William Shakespeare.

> Finis

{'input': 'What tool does ruff use to run over Jupyter Notebooks? Who wrote the most popular sonnet?',
 'output': 'The most famous sonnet is "Sonnet 18" by William Shakespeare.'}